# torch tensorss
钱箱神经网络，计算损失，反向传播

In [2]:
import torch 

N=64 # 64个输入
D_in=1000 #输入1000维
H=100 # 中间层 100维
D_out=10 #输出10维

In [8]:
# 随机创建一些训练数据
x = torch.randn(N,D_in)
y = torch.randn(N,D_out)

w1 = torch.randn(D_in,H,requires_grad=True) # tensor专门列开一个位置叫grad，这样算backword 会把grad填上
w2 = torch.randn(H,D_out,requires_grad=True)

learning_rate=1e-6

In [9]:
for it in range(500): 
    # forward pass
#     h=x.mm(w1) # N*H
#     h_relu = h.clamp(min=0) # N*H
#     y_pred = h_relu.mm(w2)  # N*D_out
    
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    #compute loss
    loss = (y_pred - y).pow(2).sum()  # computation graph计算图
    print(it,loss.item())
    
    # backword pass
    # cumpute the gradient
    # d loss/d w1 = 链式。。。
#     grad_y_pred =2.0*(y_pred-y)
#     grad_w2 = h_relu.t().mm(grad_y_pred)
#     grad_h_relu = grad_y_pred.mm(w2.t())
#     grad_h = grad_h_relu.clone()
#     grad_h[h<0]=0
#     grad_w1 = x.t().mm(grad_h)
    
    loss.backward()
    
    # tensor所有运算都是计算图，为了让tensor不让计算图占内存
    # update weights of w1 and w2
    with torch.no_grad():
        w1 -= learning_rate*w1.grad
        w2 -= learning_rate*w2.grad
        w1.grad.zero_()
        w2.grad.zero_()

0 24442182.0
1 18017100.0
2 14943171.0
3 13031245.0
4 11426987.0
5 9721112.0
6 7960073.0
7 6232003.0
8 4725415.5
9 3491608.25
10 2556210.5
11 1868798.0
12 1381075.25
13 1036788.375
14 795124.25
15 623281.875
16 499653.3125
17 408858.34375
18 340783.0
19 288508.84375
20 247478.609375
21 214638.78125
22 187817.6875
23 165544.890625
24 146794.859375
25 130802.1640625
26 117026.5
27 105066.71875
28 94625.90625
29 85452.359375
30 77356.609375
31 70187.21875
32 63811.05859375
33 58121.1015625
34 53033.43359375
35 48471.16796875
36 44378.49609375
37 40693.78515625
38 37368.171875
39 34360.328125
40 31636.92578125
41 29167.78515625
42 26924.416015625
43 24881.5
44 23018.91796875
45 21318.880859375
46 19764.314453125
47 18340.71484375
48 17036.35546875
49 15839.859375
50 14739.7802734375
51 13728.0654296875
52 12796.529296875
53 11937.958984375
54 11145.99609375
55 10414.7958984375
56 9738.666015625
57 9112.7666015625
58 8532.9736328125
59 7995.7861328125
60 7497.36669921875
61 7034.40771484375

# pytorch:nn
- 这次使用pytorch中nn这个库来构建网络。
- 用pytorch autograd来构建计算图和计算gradient
- 然后python会自动计算gradient

In [19]:
import torch.nn as nn

N=64 # 64个输入
D_in=1000 #输入1000维
H=100 # 中间层 100维
D_out=10 #输出10维

# 随机创建一些训练数据
x = torch.randn(N,D_in).to('cuda:0')
y = torch.randn(N,D_out).to('cuda:0')

model = torch.nn.Sequential(
    torch.nn.Linear(D_in,H),
    torch.nn.ReLU(),
    torch.nn.Linear(H,D_out),
)

torch.nn.init.normal_(model[0].weight)#经验
torch.nn.init.normal_(model[2].weight)

model = model.to('cuda:0')
loss_fn = nn.MSELoss(reduction='sum')

learning_rate=1e-6

for it in range(500): 
    # forward pass
#     h=x.mm(w1) # N*H
#     h_relu = h.clamp(min=0) # N*H
#     y_pred = h_relu.mm(w2)  # N*D_out
    
    y_pred = model(x) # model.forward()
    
    #compute loss
    loss = loss_fn(y_pred,y)  # computation graph计算图
    print(it,loss.item())
    
    # backword pass
    # cumpute the gradient
    # d loss/d w1 = 链式。。。
#     grad_y_pred =2.0*(y_pred-y)
#     grad_w2 = h_relu.t().mm(grad_y_pred)
#     grad_h_relu = grad_y_pred.mm(w2.t())
#     grad_h = grad_h_relu.clone()
#     grad_h[h<0]=0
#     grad_w1 = x.t().mm(grad_h)
    model.zero_grad()
    loss.backward()
    
    # tensor所有运算都是计算图，为了让tensor不让计算图占内存
    # update weights of w1 and w2
    with torch.no_grad():  # param(tensor,grad)
        for param in model.parameters():
            param -= learning_rate*param.grad
            

0 35557200.0
1 36114676.0
2 40089768.0
3 38757912.0
4 29164308.0
5 16087516.0
6 7443400.0
7 3503010.25
8 2000440.0
9 1382073.75
10 1072548.75
11 878804.0
12 738211.8125
13 628176.125
14 538972.0
15 465244.0625
16 403743.875
17 351928.3125
18 308266.5
19 271093.21875
20 239182.984375
21 211652.765625
22 187846.40625
23 167189.71875
24 149174.28125
25 133415.9375
26 119577.890625
27 107382.65625
28 96638.890625
29 87172.5
30 78764.828125
31 71282.6796875
32 64601.5859375
33 58630.69140625
34 53285.91796875
35 48502.39453125
36 44209.0703125
37 40345.98046875
38 36864.80078125
39 33723.1640625
40 30883.587890625
41 28315.44921875
42 25986.359375
43 23871.50390625
44 21948.615234375
45 20198.72265625
46 18605.1953125
47 17151.55078125
48 15824.62890625
49 14611.1748046875
50 13501.294921875
51 12485.162109375
52 11553.724609375
53 10699.3271484375
54 9914.69921875
55 9193.8076171875
56 8531.01953125
57 7921.01025390625
58 7359.09423828125
59 6841.1552734375
60 6363.5380859375
61 5922.63085

In [17]:
model[0].weight

Parameter containing:
tensor([[ 5.9931e-03,  2.5495e-02, -2.3114e-02,  ..., -2.5103e-02,
         -1.1510e-02,  3.1857e-02],
        [ 1.2912e-02, -1.4467e-02,  6.9403e-03,  ..., -6.1057e-03,
         -1.3359e-02, -2.8196e-02],
        [ 2.5674e-02,  1.0794e-02,  3.3988e-03,  ...,  2.3447e-02,
         -2.1006e-02, -2.5804e-02],
        ...,
        [ 9.1671e-03, -1.6116e-02, -9.6578e-03,  ...,  1.6271e-02,
         -2.5629e-02,  7.0179e-03],
        [ 7.1602e-03, -5.6875e-03, -6.9846e-03,  ...,  2.7993e-02,
          1.3654e-02,  1.6649e-02],
        [-1.4346e-02, -2.4755e-02, -2.2672e-05,  ..., -2.2645e-02,
         -2.0813e-02, -7.6156e-03]], device='cuda:0', requires_grad=True)

# pytorch:optim
- 这一次不在手动更新模型的weights，而是使用optim这个包来帮助我们更新参数。
- optim这个package提供了各种不同的模型优化方法：SGD+momentum，RMSProp，adam

In [3]:
import torch.nn as nn
import torch

N=64 # 64个输入
D_in=1000 #输入1000维
H=100 # 中间层 100维
D_out=10 #输出10维


# 随机创建一些训练数据
x = torch.randn(N,D_in).to('cuda:0')
y = torch.randn(N,D_out).to('cuda:0')

model = torch.nn.Sequential(
    torch.nn.Linear(D_in,H),
    torch.nn.ReLU(),
    torch.nn.Linear(H,D_out),
)

# torch.nn.init.normal_(model[0].weight)#经验 一些模型的初始化适合一些不适合
# torch.nn.init.normal_(model[2].weight)

model = model.to('cuda:0')
loss_fn = nn.MSELoss(reduction='sum')
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

for it in range(500): 
    # forward pass
#     h=x.mm(w1) # N*H
#     h_relu = h.clamp(min=0) # N*H
#     y_pred = h_relu.mm(w2)  # N*D_out
    
    y_pred = model(x) # model.forward()
    
    #compute loss
    loss = loss_fn(y_pred,y)  # computation graph计算图
    print(it,loss.item())
    
    # backword pass
    # cumpute the gradient
    # d loss/d w1 = 链式。。。
#     grad_y_pred =2.0*(y_pred-y)
#     grad_w2 = h_relu.t().mm(grad_y_pred)
#     grad_h_relu = grad_y_pred.mm(w2.t())
#     grad_h = grad_h_relu.clone()
#     grad_h[h<0]=0
#     grad_w1 = x.t().mm(grad_h)
    optimizer.zero_grad()
    loss.backward()
    
    # update weights of w1 and w2
    optimizer.step()
            

0 703.599609375
1 686.0562133789062
2 669.0096435546875
3 652.470947265625
4 636.4136962890625
5 620.8534545898438
6 605.72119140625
7 591.044189453125
8 576.7297973632812
9 562.7486572265625
10 549.1608276367188
11 535.995361328125
12 523.24072265625
13 510.86370849609375
14 498.89849853515625
15 487.27288818359375
16 475.99774169921875
17 465.00640869140625
18 454.2627258300781
19 443.79815673828125
20 433.630615234375
21 423.7554931640625
22 414.1490478515625
23 404.7812805175781
24 395.62896728515625
25 386.7212219238281
26 378.0419921875
27 369.5470275878906
28 361.2042236328125
29 353.05023193359375
30 345.07452392578125
31 337.2701416015625
32 329.6502990722656
33 322.21197509765625
34 314.94085693359375
35 307.826171875
36 300.8843994140625
37 294.0935974121094
38 287.41400146484375
39 280.84564208984375
40 274.42889404296875
41 268.16162109375
42 262.02081298828125
43 255.996337890625
44 250.08665466308594
45 244.3001708984375
46 238.62835693359375
47 233.07894897460938
48 227

In [4]:
import torch.nn as nn
import torch

N=64 # 64个输入
D_in=1000 #输入1000维
H=100 # 中间层 100维
D_out=10 #输出10维

class TwoLayerNet(torch.nn.Module):
    def __init__(self,D_in,H,D_out):
        super(TwoLayerNet,self).__init__()
        self.linear1 = torch.nn.Linear(D_in,H,bias=False)
        self.linear2 = torch.nn.Linear(H,D_out,bias=False)
        
    def forward(self,x):
        y_pred = self.linear2(self.linear1(1).clamp(min=0))
        return y_pred
    
model=TwoLayerNet(D_in,H,D_out)

# 随机创建一些训练数据
x = torch.randn(N,D_in).to('cuda:0')
y = torch.randn(N,D_out).to('cuda:0')



# torch.nn.init.normal_(model[0].weight)#经验 一些模型的初始化适合一些不适合
# torch.nn.init.normal_(model[2].weight)

model = model.to('cuda:0')
loss_fn = nn.MSELoss(reduction='sum')
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

for it in range(500): 
    # forward pass
#     h=x.mm(w1) # N*H
#     h_relu = h.clamp(min=0) # N*H
#     y_pred = h_relu.mm(w2)  # N*D_out
    
    y_pred = model(x) # model.forward()
    
    #compute loss
    loss = loss_fn(y_pred,y)  # computation graph计算图
    print(it,loss.item())
    
    # backword pass
    # cumpute the gradient
    # d loss/d w1 = 链式。。。
#     grad_y_pred =2.0*(y_pred-y)
#     grad_w2 = h_relu.t().mm(grad_y_pred)
#     grad_h_relu = grad_y_pred.mm(w2.t())
#     grad_h = grad_h_relu.clone()
#     grad_h[h<0]=0
#     grad_w1 = x.t().mm(grad_h)
    optimizer.zero_grad()
    loss.backward()
    
    # update weights of w1 and w2
    optimizer.step()
            

AttributeError: 'int' object has no attribute 'dim'